In [1]:
%cd /home/soda/rcappuzz/work/prepare-data-lakes

/home/soda/rcappuzz/work/prepare-data-lakes


In [2]:
import src.yago.utils as utils

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import polars as pl
from pathlib import Path
import numpy as np
from tqdm import tqdm

sns.set_theme("paper", style="whitegrid")

In [38]:
cfg = pl.Config()
cfg.set_fmt_str_lengths(150)

polars.config.Config

In [4]:
yago_path = Path("/storage/store3/work/jstojano/yago3/")
facts1_path = Path(yago_path, "facts_parquet/yago_updated_2022_part1")
facts2_path = Path(yago_path, "facts_parquet/yago_updated_2022_part2")

In [5]:
fname = "yagoTypes"
yagotypes_path = Path(facts1_path, f"{fname}.tsv.parquet")
df_types = utils.import_from_yago(yagotypes_path)

In [16]:
fname = "yagoFacts"
yagofacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagofacts = utils.import_from_yago(yagofacts_path)
fname = "yagoLiteralFacts"
yagoliteralfacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagoliteralfacts = utils.import_from_yago(yagoliteralfacts_path)
fname = "yagoDateFacts"
yagodatefacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagodatefacts = utils.import_from_yago(yagodatefacts_path)

In [17]:
yagofacts = yagofacts.drop("num_object")

In [19]:
yagodatefacts = yagodatefacts.with_columns(
    pl.col("cat_object")
    .str.split("^^").list.first().str.to_datetime(strict=False).dt.date().cast(pl.Utf8).alias("cat_object")
).drop_nulls("cat_object").drop("num_object")

In [20]:
yagoliteralfacts = yagoliteralfacts.with_columns(
    pl.when(pl.col("num_object").is_not_null())
    .then(pl.col("num_object"))
    .otherwise(pl.col("cat_object"))
    .alias("cat_object")
).drop("num_object")

In [23]:
df_facts = pl.concat([yagofacts, yagoliteralfacts, yagodatefacts]).drop("id")

In [40]:
df_types.filter(pl.col("cat_object").str.starts_with("<wordnet_")).unique("cat_object")

id,subject,predicate,cat_object,num_object
str,str,str,str,f64
"""<id_PP4Fgwd3O6_KCM_2WuexOboD1>""","""<South_Parade_Pier>""","""rdf:type""","""<wordnet_pier_103933529>""",null
"""<id_S?cBQTdyTg_KCM_N?8Ck7Lpgi>""","""<1988_United_States_Senate_election_in_New_York>""","""rdf:type""","""<wordnet_election_100181781>""",null
"""<id_e1rrdF7bPy_KCM_8YKdPoy?Yj>""","""<Fregat>""","""rdf:type""","""<wordnet_phase_115290337>""",null
"""<id_3eosUH!UpW_KCM_RCvUzF20tg>""","""<CODESYS>""","""rdf:type""","""<wordnet_platform_103961939>""",null
"""<id_wWPNkb?tcA_KCM_gcYrxUIzku>""","""<Penny_Blue>""","""rdf:type""","""<wordnet_cast_113913566>""",null
"""<id_4!vVb!QsLw_KCM_rCmmppklcW>""","""<1951_Italian_Grand_Prix>""","""rdf:type""","""<wordnet_race_107472657>""",null
"""<id_GMDh?X49lw_KCM_HsZFfsa9!W>""","""<Alkaline_water_electrolysis>""","""rdf:type""","""<wordnet_electrolysis_113472125>""",null
"""<id_WiEDcDfNKE_KCM_l80omYS1aQ>""","""<Tehran_Peace_Museum>""","""rdf:type""","""<wordnet_museum_103800563>""",null
"""<id_lPkWJz6mRm_KCM_hOycGJ29OP>""","""<West_Division_(CFL)>""","""rdf:type""","""<wordnet_division_108213205>""",null
